In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI实验: 自定义训练自动记录 - 本地脚本

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_custom_training_autologging_local_script.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_custom_training_autologging_local_script.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/get_started_with_custom_training_autologging_local_script.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/>

注意：此笔记本已在以下环境中进行了测试：

* Python 版本= 3.9

## 概述

作为一名实验大型模型的数据科学家，您需要一种在可扩展的训练服务上运行实验以记录参数和指标的方式。这可以保证可复现性。

使用 Vertex AI 训练和 Vertex AI Experiments 自动记录集成，您可以使用 `enable_autolog` 参数在规模上运行您的 ML 实验，并自动记录它们的参数和指标。

了解更多关于[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments)以及如何[为实验运行自动记录数据](https://cloud.google.com/vertex-ai/docs/experiments/autolog-data)。

### 目标

在本教程中，您将学习如何通过利用与Vertex AI实验的集成，自动记录在Vertex AI Training上运行的ML实验的参数和度量。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI实验
- Vertex AI培训

执行的步骤包括：

- 在脚本中 formalize 模型实验
- 在Vertex AI Training上使用本地脚本运行模型训练
- 在Vertex AI实验中查看ML实验参数和度量

###数据集

[玻璃鉴定数据集](https://archive-beta.ics.uci.edu/dataset/42/glass+identification)是来自美国法庭科学服务的数据集，共包含6种不同类型的玻璃，其定义基于其氧化物含量（例如Na、Fe、K等）。该数据集的目标是根据氧化物特征对玻璃的类型进行分类。

### 费用

本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，
以及 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，
并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成费用估算。

## 安装

安装以下所需的软件包以执行此笔记本。

In [ ]:
# Install the packages
USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform --quiet --no-warn-conflicts

只有协作：取消注释以下单元格以重启内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。创建账户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保您的项目已经启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的“REGION”变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### UUID
如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源之间出现名称冲突，请为每个实例会话创建一个通用唯一标识符（UUID）。在此教程中创建的资源名称后附加UUID。

In [ ]:
import random
import string


# Generate a uuid of length 8
def generate_uuid():
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=8))


UUID = generate_uuid()

### 认证您的Google云账户

根据您的Jupyter环境，您可能需要手动进行认证。请按照以下相关说明操作。

**1. Vertex AI工作台**
* 无需操作，因为您已经通过身份验证。

2. 本地 JupyterLab 实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 设置服务账号

**如果您不知道您的服务账号**，请尝试使用 `gcloud` 命令在下方执行第二个单元格来获取您的服务账号。

*注意:* 自动查找您的服务账号的代码适用于用户管理的Workbench AI笔记本。如果您使用的是完全托管的笔记本，您需要手动输入您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
IS_COLAB = False

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI训练设置服务账号访问权限

运行以下命令来授权您的服务账号在自定义训练作业运行时读取和更新Vertex AI ML Metadata中的元数据 -- 您只需针对每个服务账号运行一次这些命令。查看[文档](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-training#create_a_service_account_with_required_permissions)获取更多信息。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 创建项目模板
设置您在本教程中使用的文件夹。

In [ ]:
import os

TUTORIAL_DIR = os.path.join(
    os.getcwd(), "custom_training_autologging_local_script_tutorial"
)
os.makedirs(TUTORIAL_DIR, exist_ok=True)

### 获取数据集
从公共云存储桶中获取玻璃分类数据集。

In [ ]:
SOURCE_DATA_URL = "gs://cloud-samples-data/vertex-ai/dataset-management/datasets/uci_glass_preprocessed/glass.csv"
DESTINATION_DATA_URL = f"{BUCKET_URI}/data/glass.csv"

! gsutil cp $SOURCE_DATA_URL $DESTINATION_DATA_URL

### 导入库

In [ ]:
import os

from google.cloud import aiplatform as vertex_ai

###定义常量

In [ ]:
# Training
EXPERIMENT_NAME = f"glass-classification-{UUID}"
TRAIN_SCRIPT_PATH = os.path.join(TUTORIAL_DIR, "task.py")
JOB_DISPLAY_NAME = f"sklearn-autologged-custom-job-{UUID}"
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    f"{REGION.split('-')[0]}-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest"
)
MODEL_FILE_URI = f"{BUCKET_URI}/models/model.joblib"
DESTINATION_DATA_PATH = DESTINATION_DATA_URL.replace("gs://", "/gcs/")
MODEL_FILE_PATH = MODEL_FILE_URI.replace("gs://", "/gcs/")
REPLICA_COUNT = 1
TRAIN_MACHINE_TYPE = "n1-standard-4"
TRAINING_JOBS_URI = f"{BUCKET_URI}/jobs"

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 创建一个用于跟踪训练参数和指标的实验

首先，使用 `init()` 方法初始化一个实验。

In [ ]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)

使用预构建容器训练一个 scikit-learn 模型，然后使用预构建容器为 scikit-learn 模型训练一个自定义模型。

创建scikit-learn训练脚本

In [ ]:
task_script = f"""
#!/usr/bin/env python3

'''
A simple module to train a classifier on the glass dataset.
'''

# Libraries
import argparse
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib


# Variables
DATA_PATH = '{DESTINATION_DATA_PATH}'
MODEL_PATH = '{MODEL_FILE_PATH}'
TEST_SIZE = 0.2
SEED = 8

# Helpers
def read_data(path):
    df = pd.read_csv(path)
    return df


def split_data(df):
    y = df.pop('glass_type')
    X = df
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=SEED)
    return X_train, X_test, y_train, y_test


def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=5)
    model.fit(X_train, y_train)
    return model


def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


def save_model(model, path):
    p = Path(path)
    if not p.parent.exists():
      p.parent.mkdir(parents=True)
    joblib.dump(model, path)


def main():

    # Read data
    df = read_data(DATA_PATH)

    # Split data
    X_train, X_test, y_train, y_test = split_data(df)

    # Train model
    model = train_model(X_train, y_train)

    # Evaluate model
    accuracy = evaluate_model(model, X_test, y_test)
    print('Model accuracy:', accuracy)

    # Save model
    save_model(model, MODEL_PATH)



if __name__ == '__main__':

    # Run main
    main()
"""

with open(TRAIN_SCRIPT_PATH, "w") as train_file:
    train_file.write(task_script)
train_file.close()

定义自定义培训作业

使用预构建的容器镜像定义一个自定义作业，其中包装了作为 Python 脚本的训练代码。在这种情况下，您设置 `enable_autolog=True`，以在培训作业完成后自动跟踪参数和指标。

In [ ]:
job = vertex_ai.CustomJob.from_local_script(
    project=PROJECT_ID,
    staging_bucket=TRAINING_JOBS_URI,
    display_name=JOB_DISPLAY_NAME,
    script_path=TRAIN_SCRIPT_PATH,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
    requirements=["pandas", "scikit-learn"],
    replica_count=REPLICA_COUNT,
    machine_type=TRAIN_MACHINE_TYPE,
    enable_autolog=True,
)

### 运行自定义训练任务
接下来，使用 `run` 方法运行训练任务。

In [ ]:
job.run(experiment=EXPERIMENT_NAME, service_account=SERVICE_ACCOUNT)

获取您的自动记录实验

在训练模型后，您可以获取自动记录实验的参数和指标。

In [ ]:
experiment_df = vertex_ai.get_experiment_df(experiment=EXPERIMENT_NAME)
experiment_df.T

您还可以获取与您运行的实验相关联的自定义培训作业元数据。您可以恢复已记录的实验，并使用`get_logged_custom_jobs()`来获取与该实验运行相关联的所有`CustomJobs`资源。然后，您可以使用`job_spec`来打印自定义作业元数据，例如训练Python软件包、训练资源等。

In [ ]:
experiment_run = experiment_df.run_name.iloc[0]

with vertex_ai.start_run(experiment_run, resume=True) as run:
    # get the latest logged custom job
    logged_job = run.get_logged_custom_jobs()[-1]

print(logged_job.job_spec)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

# delete flags
delete_experiment = False
delete_bucket = False

# Delete experiment
if delete_experiment or os.getenv("IS_TESTING"):
    experiment = vertex_ai.Experiment.get(experiment_name=EXPERIMENT_NAME)
    experiment.delete(delete_backing_tensorboard_runs=True)

# Delete Cloud Storage objects that were created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI